In [32]:
import hipscat_import.pipeline as runner
from hipscat_import.index.arguments import IndexArguments
import numpy as np
import dask
import pyarrow.parquet as pq
import pyarrow as pa

dask.config.set({"dataframe.convert-string": False})

In [11]:
from dask.distributed import Client

client = Client(
        local_directory="/data3/epyc/data3/hipscat/tmp/",
        n_workers=12,
        threads_per_worker=1,
    )
client

/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39158 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:39158/status,
Dashboard: http://127.0.0.1:39158/status,Workers: 12
Total threads: 12,Total memory: 0.98 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39679,Workers: 12
Dashboard: http://127.0.0.1:39158/status,Total threads: 12
Started: Just now,Total memory: 0.98 TiB
Comm: tcp://127.0.0.1:38075,Total threads: 1
Dashboard: http://127.0.0.1:32918/status,Memory: 83.97 GiB
Nanny: tcp://127.0.0.1:44155,


In [2]:
global_min= 4295806720
global_max= 6917528997577384320
num_row_groups =3933

increment = int((global_max-global_min)/num_row_groups)

divisions = np.append(np.arange(start = global_min, stop = global_max, step = increment), global_max)
divisions = divisions.tolist()

args = IndexArguments(
    input_catalog_path="/data3/epyc/data3/hipscat/catalogs/gaia_dr3/gaia",
    indexing_column="source_id",
    output_path="/data3/epyc/data3/hipscat/catalogs/gaia_dr3/",
    output_artifact_name="gaia_source_id_hc_index",
    include_hipscat_index=True,
    include_order_pixel=False,
    compute_partition_size=2_000_000_000,
    division_hints=divisions,
    drop_duplicates=False,
    overwrite=True,
    completion_email_address="delucchi@andrew.cmu.edu",
)
runner.pipeline_with_client(args, client)

/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39725 instead
  warnings.warn(


Finishing :   0%|          | 0/4 [00:00<?, ?it/s]

In [34]:
single_file = "/data3/epyc/data3/hipscat/raw/gaia/epoch_parquet/epoch_data_786097-786431.parquet"

parquet_file = pq.ParquetFile(single_file)
schema = parquet_file.schema.to_arrow_schema()
schema = schema.insert(0, pa.field("_hipscat_index", pa.uint64()))
schema

_hipscat_index: uint64
solution_id: int64
source_id: int64
n_transits: int16
transit_id: list<element: int64>
  child 0, element: int64
g_transit_time: list<element: double>
  child 0, element: double
g_transit_flux: list<element: double>
  child 0, element: double
g_transit_flux_error: list<element: double>
  child 0, element: double
g_transit_flux_over_error: list<element: float>
  child 0, element: float
g_transit_mag: list<element: double>
  child 0, element: double
g_transit_n_obs: list<element: int8>
  child 0, element: int8
bp_obs_time: list<element: double>
  child 0, element: double
bp_flux: list<element: double>
  child 0, element: double
bp_flux_error: list<element: double>
  child 0, element: double
bp_flux_over_error: list<element: float>
  child 0, element: float
bp_mag: list<element: double>
  child 0, element: double
rp_obs_time: list<element: double>
  child 0, element: double
rp_flux: list<element: double>
  child 0, element: double
rp_flux_error: list<element: double

In [28]:
left_table = dd.read_parquet(
        path="/data3/epyc/data3/hipscat/catalogs/gaia_dr3/gaia_source_id_hc_index",
        engine="pyarrow",
        filesystem="arrow",
    )

In [29]:


right_table = dd.read_parquet(
        path=single_file,
        engine="pyarrow",
        filesystem="arrow",
    )

,solution_id,source_id,n_transits,transit_id,g_transit_time,g_transit_flux,g_transit_flux_error,g_transit_flux_over_error,g_transit_mag,g_transit_n_obs,...,photometry_flag_af5_reject,photometry_flag_af6_reject,photometry_flag_af7_reject,photometry_flag_af8_reject,photometry_flag_af9_reject,photometry_flag_bp_reject,photometry_flag_rp_reject,variability_flag_g_reject,variability_flag_bp_reject,variability_flag_rp_reject
0,375316653866487564,6914582993610503168,51,"[20017163183592008, 21758024825582601, 2176211...","[1758.8449968394293, 1790.324541228065, 1790.3...","[12062.938251410498, 15253.480085674319, 14509...","[412.20452777418814, 122.89795899792132, 386.1...","[29.264448, 124.115, 37.57194, 19.336327, 22.4...","[15.483734105225322, 15.22894451857305, 15.283...","[9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 9, 8, 8, 9, ...",...,"[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru..."
1,375316653866487564,6914583165409200896,52,"[20017163826369356, 21758024488465806, 2176211...","[1758.8450063936607, 1790.324533233702, 1790.3...","[97685.41265966844, 98156.55958142849, 98277.4...","[77.21957508423984, 54.279022447691816, 88.895...","[1265.0343, 1808.3701, 1105.5352, 714.08246, 8...","[13.21279257839338, 13.207568547489927, 13.206...","[9, 9, 8, 9, 9, 9, 8, 9, 9, 9, 9, 9, 8, 9, 8, ...",...,"[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru..."
2,375316653866487564,6914583886965297408,51,"[20017166380045239, 21758024853889773, 2176211...","[1758.8450475922664, 1790.3245357298917, 1790....","[3028.324850630355, 2835.5809252715994, 3389.8...","[12.237938939909453, 8.85288918420613, 11.5744...","[247.45383, 320.30005, 292.8713, 232.69237, 20...","[16.984360716018344, 17.05576175149253, 16.861...","[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ...",...,"[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru..."
3,375316653866487564,6914584672941958656,59,"[20017161687147720, 21762119308210553, 2275763...","[1758.8449984137585, 1790.3985883447478, 1808....","[890.335027775967, 898.3316286885513, 898.3595...","[7.0017054483115775, 6.647580970816504, 9.4710...","[127.15974, 135.13663, 94.8537, 112.76634, 127...","[18.313483217478623, 18.303775140303483, 18.30...","[8, 9, 9, 9, 9, 9, 8, 8, 8, 9, 9, 8, 9, 8, 8, ...",...,"[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[True, True, True, True, True, True, True, Tru...","[Tru

In [30]:
%%time
result = left_table.merge(right_table, how="right", left_index=True, right_on="source_id")

CPU times: user 83.7 ms, sys: 13.6 ms, total: 97.3 ms
Wall time: 93 ms


In [35]:
%%time

# result = result.repartition(npartitions=1)

result.to_parquet("/data3/epyc/data3/hipscat/raw/gaia/epoch_w_hipscat/epoch_data_786097-786431.parquet",
                  engine="pyarrow",
                  schema=schema,
                  write_index=False)

CPU times: user 32.4 s, sys: 12.4 s, total: 44.9 s
Wall time: 2min 43s


## But does it import?


In [36]:
from hipscat_import.catalog.arguments import ImportArguments

input_file_list=["/data3/epyc/data3/hipscat/raw/gaia/epoch_w_hipscat/epoch_data_786097-786431.parquet/part.0.parquet"]

args = ImportArguments(
    output_path="/data3/epyc/data3/hipscat/test_catalogs/",
    output_artifact_name="gaia_epoch_1",
    input_file_list=input_file_list,
    file_reader="parquet",
    use_hipscat_index=True,
    pixel_threshold=1_000_000,
    tmp_dir="/data3/epyc/data3/hipscat/tmp/",
    highest_healpix_order=8,
    overwrite=True,
    completion_email_address="delucchi@andrew.cmu.edu",
)
runner.pipeline_with_client(args, client)

Planning  :   0%|          | 0/5 [00:00<?, ?it/s]

Mapping   :   0%|          | 0/1 [00:00<?, ?it/s]

Binning   :   0%|          | 0/2 [00:00<?, ?it/s]

Splitting :   0%|          | 0/1 [00:00<?, ?it/s]

Reducing  :   0%|          | 0/1 [00:00<?, ?it/s]

Finishing :   0%|          | 0/5 [00:00<?, ?it/s]